### Documentation de la librairie Jupylet


[Documentation Jupylet](https://jupylet.readthedocs.io/en/latest/)



### Prérequis


Il faut utiliser la librairie Python 3.9.13 et non la dernière version 3.10 car la librairie jupylet n'est pas compatible avec la version Python 3.10.
Si la version 3.10 ou plus est installé sur votre poste, il faut procéder à la désinstallation et effacer toute référence à des fichiers \AppData\Local\Programs\Python\Python310 par exemple ainsi que les variables d'environnements reliés à la version de Python 3.10.


Pour exécuter le code ci-joint, il faut installer la librairie jupylet

Voici le guide d'installation

[Guide d'installation](https://jupylet.readthedocs.io/en/latest/programmers_reference_guide/getting_started.html)


```pip install jupylet```

Ensuite il faut procéder à la post-installation.

```python -m jupylet postinstall```

### Pong

Exécutez ce programme et regardez ce qui se passe.


Vous pouvez exécuter le programme dans le fichier StemPongComplet.ipynb

Cliquer la souris dans le l'image du jeu.  Pour bouger les pallettes il faut utiliser les touches flèche gauche et flèche droite pour une palette ainsi que la touche A et D pour l'autre palette


* le son est fournis par  [freesound](https://freesound.org/people/NoiseCollector/sounds/4359/).
* Les fonts de Commodore 64 sont fournis par  [KreativeKorp](https://www.kreativekorp.com/software/fonts/c64.shtml).

Ce jeu est fournis par le site web https://jupylet.readthedocs.io/en/latest/index.html




# Importation des librairies

In [ ]:
import logging
import sys
import os

import numpy as np

In [ ]:
sys.path.insert(0, os.path.abspath('./..'))

Ici on ajoute les librairies de code pour nous permettre d'avoir des fonctionnalités tel que la capacité d'écrire sur l'écran (jupylet.app), d'écrire du texte (jupylet.label), d'afficher des petits morceaux d'image (jupylet.sprite), d'ajouter du son à notre jeu (jupylet.audio.sample)

In [ ]:
import jupylet.color                                # Composant permettant de definir des couleurs

from jupylet.app import App                         # Composant de l'application Jupylet
from jupylet.state import State                     # Objet permettant de conserver les états de différentes variables
from jupylet.label import Label                     # Composant de la librairie jupylet permettant d'afficher des textes dans le jeu
from jupylet.sprite import Sprite                   # Composant de la librairie jupylet permettant d'afficher des images dans le jeu

from jupylet.audio.sample import Sample             # Composant de la librairie jupylet permettant de jouer des fichiers de son

In [ ]:
app = App()#log_level=logging.INFO)     # On crée l'objet application qui nous permettra de mettre nos animations dedans.  

# Création de nos objets du jeu

Ici on défini les couleurs de notre fond d'écran et de l'avant plan ..

In [ ]:
background = '#3e32a2'         # couleur d'arrière plan centrale
foreground = '#7c71da'         # couleur des contours et resultat.

Plus bas on affecte une matrice de 32 * 32 de chiffre 1. Ceci est pour créer une image avec 32 par 32 pixels 

![](.\images\matrix-example2.png)

Par contre la matrice a0 est de grandeur de 32 ligne par 32 colonnes, ce serait assez long ajouter toutes les colonnes et les lignes dans le dessin.

Ensuite on multiplie la matrice par 255, et voilà ce que ça donnera ...

![](.\images\matrix-example3.png)

Encore là, il faut compter 32 lignes et 32 colonnes pour la variable a0

Pour la variable a1, on aurra aussi la valeur 255 dans chacune des cases, cette matrice aura 128 lignes et 16 colonnes qui correspond à la taille en pixel d'une palette.

Enfin la variables a2 est un peu plus compliquée, 

Au lieu d'avoir une matrice, 2 par 2, on aura un cube comme un cube Rubix.

Les trois axes (X, Y, Z) auront différentes valeurs.  
Par example l'axe X sera relié à la largeur 
L'axe Y sera relié à la hauteur de l'application 
L'axe Z représente une profondeur, dans notre cas, c'est pour accumuler la couleur à une position précise.

![](.\images\cube.png)

L'image représente un cube de 5 par 5 par 5.  Dans notre case c'est 90% de la hauteur de l'application par 90% de la largeur de l'application en pixel par 3 (pour contenir le code de couleur à 3 Bytes (code de couleur RGB '#7c71da')).

Et voici les différentes représentations d'une des données, sous forme de vecteur, de matrice, ou de matrice multi-dimensionnel comme dans le cas de la variable a2.

![](.\images\arrayMatrix.png)



In [ ]:
a0 = np.ones((32, 32)) * 255                                        # génération d'une image avec 32 par 32 pixels pour la balle
a1 = np.ones((128, 16)) * 255                                       # génération d'une image 128 par 16 pixels pour les palettes
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255  # génération des pixels de l'image du fond d'écran, le fond d'écran correspond à 9/10 du jeu, donc le 1/10 de chaque côté sera la bordure, le chiffre 3 a la fin permettra d'ajouter la couleur sur chacune des positions du tableau. 

A cet endroit on définit des images en leur donnant la grandeur qu'on les veut.

Par exemple, **ball** sera la balle qui se déplace.
**padl** et **padr** seront les palettes qui se déplacent pour intercepter la balle.

Nous donnons les coordonnées selons les tableaux que nous avons fait préalablement pour indiquer à nos images (Sprite) les positions disponibles dans le notre jeu, ça nous permettra de déplacer les images en vertical (X) et horizontal (y)

field est notre terrain de jeu, l'arrière plan du jeu.

Pour définir l'image, on indique la position dans la variable **x** et **y** comme par exemple, les pallettes **padl** et **palr** sont situés soit près à gauche **48** pixels ou à la droite (**app.width - 48**) pixels

![](.\images\paddle.png)


Pour ce qui est de l'arrière plan du jeu, revenons à l'instruction précédente 
**a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255**

Nous indiquons dans le tableau que nous voulons remplir 90% du tableau en hauteur et en largeur, la troisième dimension du tableau permet de conserver la couleur.

Ceci donnera l'effet d'avoir le tour du canvas (10%) avec la couleur de fond. Pour celui-ci on le centre en **x** et **y** dans notre canevas. 

In [ ]:
ball = Sprite(a0, y=app.height/2, x=app.width/2)    # image de la balle

padl = Sprite(a1, y=app.height/2, x=48)             # image de la palette de gauche
padr = Sprite(a1, y=app.height/2, x=app.width-48)   # image pour la palette de droite 

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background)  # image pour l'image de font, l'image mesure 90% du jeu (voir variable a2) et est centré dans l'application, on ajoute la couleur de background pour le sprite 

Comme tout bon jeu, on doit ajouter du son, et pour ça, on lit un fichier de son.  Comme lorsque nous allons dans une application pour écouter de la musique nous choisissons un fichier qui nous fait jouer de la musique.

In [ ]:
pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()  # chargement du son

Tout bon jeu à aussi un résultat, il faut l'afficher, donc on utilise un **Label** qui nous permet de visualiser du texte dans notre jeu.  On lui indique des paramètres comme la taille, la position dans notre jeu, le type de lettre.
Le type de lettre est nommé font:
Voici une liste de polices de caractère(fonts) déjà existants qu'on peut utiliser pour écrire notre texte dans notre jeu.

On peut remarquer que les positions sont centrées en vertical **app.height / 2** et horizontal, l'une est positionné à 64 pixels du bord du jeu et l'autre à la droite décalé de 64 pixels, la valeur initiale des **Label** est 0, bien sûr personne n'a encore marqué un point.

Exemple de **fonts**

![](.\images\font.png)

In [ ]:
scorel = Label(
    '0', font_size=42, color=foreground,       # etiquette du résultat à gauche
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground,      #  etiquette de résultat à droite 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)

# On dessine

Ensuite, il faut bien afficher nos images et notre texte, on utilise la commande **draw** de chacun des textes et des images pour que ceux-ci puissent s'afficher. 

Le premier **field** est le plan de jeu
**scorel** et **scorer** sont les résultats de chacun des joueurs (celui à gauche et celui à droite)

Ensuite on dessine la balle **ball.draw()**

Enfin on dessine les 2 palettes **padl** et **padr**

In [ ]:
@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)  # on efface tout
    
    field.draw()                        # on dessine le fond du jeu
     
    scorel.draw()                       # on dessine le résultat à gauche
    scorer.draw()                       # on dessine le résultat de droite
    
    ball.draw()                         # on dessine la balle
    padl.draw()                         # on dessine la palette à gauche
    padr.draw()                         # on dessine la palette de droite

#  Résultat d'affichage du jeu sans animation

![](.\images\noanimation.png)

# Etape 1: Affichage des objets

In [ ]:
import logging
import sys
import os

import numpy as np


sys.path.insert(0, os.path.abspath('./..'))

import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

app = App()#log_level=logging.INFO)

background = '#3e32a2'
foreground = '#7c71da'

a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255


pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()

ball = Sprite(a0, y=app.height/2, x=app.width/2)

padl = Sprite(a1, y=app.height/2, x=48)
padr = Sprite(a1, y=app.height/2, x=app.width-48)

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

scorel = Label(
    '0', font_size=42, color=foreground, 
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground, 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)


@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()

app.run()

# Initialisation de l'état du jeu

### Différentes variables sur le jeu
![](.\images\noanimation2.png)


C'est bien d'afficher nos palettes et notre balle mais si elles ne bouge pas alors notre jeu sera très ennuyant.  C'est pour ça qu'on doit ajouter du mouvement et connaître la position de notre balle et de nos pallettes à tout moment pour savoir comment réagir au déplacement de la balle. Ainsi que les touches appuyées par l'utilisateur.

Par exemple lorsque le joueur appuie sur la flèche gauche (left) notre variable d'état **left** sera à vrai (true) et la même chose pour la variable **right** sera à vraie lorsque sa touche sera appuyée, sinon elle sera à **false**.


Maintenant il faut assigner au début du jeu des variables que nous aurons besoin pendant le jeu.


La variable **pyl** permet de regarder la position de la palette à gauche au centre de la palette en vertical, elle est assignée initialement au centre vertical du jeu.

La variable **pyr** elle aussi conserve la position centrale en vertical mais cette fois de la palette à droite,  elle est assignée initialement au centre vertical du jeu.

Les variables **vyl** et **vyr** sont les variables qui conservent la vitesse des palettes et sont assignés à 250 au début et cette vitesse restera constante.

Les variables **left, right, key_a et key_d** permettent de savoir si la touche est appuyée ou non.  Elle sont assignées à **false** au début car aucune touche n'est appuyée 

Les varaibles **bvx** et **bvy** permettent de conserver la vitesse de la balle en vertical **bvx** et en horizontal **bvy**.  Arbitrairement la vitesse initiale à été choisie à 192 au départ.

Les variables **sl** et **sr** conserve le résultat des joueurs de gauche **sl** et de droite **sr**, elles sont assignées à 0 car elles n'ont aucun pointage au début.



In [ ]:
state = State(
    
    sl = 0,                             # résultat de gauche
    sr = 0,                             # résultat de droite
    
    bvx = 192,                          # vitesse de la balle horizontale
    bvy = 192,                          # vitesse de la balle verticale
    
    vyl = 250,                          # vitesse de la palette de gauche en vertical
    pyl = app.height/2,                 # position de la palette de gauche 

    vyr = 250,                          # vitesse de la palette de droite en vertical
    pyr = app.height/2,                 # position de la palette de droite en vertical

    left = False,                       # flèche gauche appuyée on non
    right = False,                      # flèche droite appuyée on non

    key_a = False,                      # touche A appuyée ou non
    key_d = False,                      # touche D appuyée ou non
)


# Etat des touches du clavier

Ici on vérifie si une touche est appuyée ou non, on reçoit un événement **keys.ACTION_PRESS** et **keys.ACTION_RELEASE** qui indiquent respectivement, si la touche est enfoncée ou non.  On ajuste alors les variables d'état en conséquence **left, right, key_a, key_d**

In [ ]:
@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:     # événement d'une touche appuyée
        
        if key == keys.LEFT:            # flèche gauche
            state.left = True

        if key == keys.RIGHT:           # flèche droite
            state.right = True

        if key == keys.A:               # touche A
            state.key_a = True

        if key == keys.D:               # touche D
            state.key_d = True

    if action == keys.ACTION_RELEASE:   # événement d'une touche libérée

    
        if key == keys.LEFT:            # flèche gauche
            state.left = False

        if key == keys.RIGHT:           # flèche droite
            state.right = False

        if key == keys.A:               # touche A
            state.key_a = False

        if key == keys.D:               # touche D
            state.key_d = False

# Le déplacement des palettes 

La fonction qui suit est appelée à chaque 1/60 ième de seconde.  Donc c'est comme une horloge qui appelle la fonction.  L'objectif de la fonction est d'évaluer la position des deux palettes à des moments précis pour les dessiner aux endroits appropriés.

    
Si une des touches est enfoncée, on associe la direction de la vitesse positive ou négative dans le dernier 1/60 ième de seconde **moveL ou moveR** soit en valeur positive vers le haut, soit en valeur négative vers le bas.

### Calcul de la nouvelle position de la palette du dernier laps de temps (1/60) de seconde

La vitesse dans ce jeu est constante, et on a donner une valeur arbitraire de 250 pixels par secondes

Lorsqu'une touche est appuyée, nous conservons l'état du déplacement soit dans la variable **moveR** pour la palette de droite ou **moveL** pour la palette de gauche.

Les directions de mouvement sont : 
* -1 : vers le bas
* 0 : en arrêt 
* +1 vers le haut

Une fois que nous avons les mouvements des deux palettes, il s'agit d'additionner 1/60 de la vitesse dans la direction positive ou négative additionnée de la vitesse précédente.  Ceci donnera notre nouveau déplacement de la palette. 

**padl.y += moveL * state.vyl * dt**
    

A ce moment, on protège le fait de ne pas dessiner les palettes en dehors du cadre du jeu.

```python
    padr.clip_position(app.width, app.height)
    padl.clip_position(app.width, app.height)   
```



In [ ]:
@app.run_me_every(1/60)
def update_pads(ct, dt):
    moveL = 0                                                 # Initialisation du deplacement de la palette gauche
    moveR = 0                                                 # Initialisation du deplacement de la palette droite   
    if state.right:
        moveR = 1                                             # Déplacement positif de la palette droite (vers vers le haut)  
        
    if state.left:
        moveR = -1                                            # Déplacement negatif de la palette droite (vers le bas)  

    if state.key_a:
        moveL = 1                                             # Déplacement positif de la palette gauche (vers le haut)  

    if state.key_d:
        moveL = -1                                            # Déplacement negatif de la palette droite (vers le bas)  

    if moveL != 0:
        padl.y += moveL * state.vyl * dt                      # calcul de la position de la palette gauche
    if moveR != 0:
        padr.y += moveR * state.vyr * dt                      # calcul de la position de la palette droite
          
    
    padr.clip_position(app.width, app.height)                 # Conserver l'image de la palette droite dans le cadre du jeu
    padl.clip_position(app.width, app.height)                 # Conserver l'image de la palette gauche dans le cadre du jeu



# Etape 2 : Jeu avec déplacement des palettes

In [1]:
import logging
import sys
import os

import numpy as np


sys.path.insert(0, os.path.abspath('./..'))

import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

app = App()#log_level=logging.INFO)

background = '#3e32a2'
foreground = '#7c71da'

a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255


pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()

ball = Sprite(a0, y=app.height/2, x=app.width/2)

padl = Sprite(a1, y=app.height/2, x=48)
padr = Sprite(a1, y=app.height/2, x=app.width-48)

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

scorel = Label(
    '0', font_size=42, color=foreground, 
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground, 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)


@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()


state = State(
    
    sl = 0,                             # résultat de gauche
    sr = 0,                             # résultat de droite
    
    bvx = 192,                          # vitesse de la balle horizontale
    bvy = 192,                          # vitesse de la balle verticale
    
    vyl = 250,                          # vitesse de la palette de gauche en vertical
    pyl = app.height/2,                 # position de la palette de gauche 

    vyr = 250,                            # vitesse de la palette de droite en vertical
    pyr = app.height/2,                 # position de la palette de droite en vertical

    left = False,                       # flèche gauche appuyée on non
    right = False,                      # flèche droite appuyée on non

    key_a = False,                      # touche A appuyée ou non
    key_d = False,                      # touche D appuyée ou non
)


@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:
        
        if key == keys.LEFT:
            state.left = True

        if key == keys.RIGHT:
            state.right = True

        if key == keys.A:
            state.key_a = True

        if key == keys.D:
            state.key_d = True

    if action == keys.ACTION_RELEASE:

    
        if key == keys.LEFT:
            state.left = False

        if key == keys.RIGHT:
            state.right = False

        if key == keys.A:
            state.key_a = False

        if key == keys.D:
            state.key_d = False


@app.run_me_every(1/60)
def update_pads(ct, dt):
    moveL = 0                                                 # Initialisation du deplacement de la palette gauche
    moveR = 0                                                 # Initialisation du deplacement de la palette droite   
    if state.right:
        moveR = 1                                             # Déplacement positif de la palette droite (vers vers le haut)  
        
    if state.left:
        moveR = -1                                            # Déplacement negatif de la palette droite (vers le bas)  

    if state.key_a:
        moveL = 1                                             # Déplacement positif de la palette gauche (vers le haut)  

    if state.key_d:
        moveL = -1                                            # Déplacement negatif de la palette droite (vers le bas)  

    if moveL != 0:
        padl.y += moveL * state.vyl * dt                      # calcul de la position de la palette gauche
    if moveR != 0:
        padr.y += moveR * state.vyr * dt                      # calcul de la position de la palette droite
          
    
    padr.clip_position(app.width, app.height)                 # Conserver l'image de la palette droite dans le cadre du jeu
    padl.clip_position(app.width, app.height)                 # Conserver l'image de la palette gauche dans le cadre du jeu



app.run()


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

# Déplacement de la balle 

Maintenant c'est le tour de la balle,  
Il faut déplacer la balle dans un intervalle précis, encore une fois tous les 1 soixantième de secondes.

Ici on conserve la composant vitesse cumulée de la balle (horizontale et verticale) pour une comparaison future.

```python
    bs0 = state.bvx ** 2 + state.bvy ** 2
```
Comme la balle qui est un carré tourne toujours, On calcule une portion de la rotation de l'angle selon le temps (1/60) donc l'angle sera 200 degré par seconde, on prend une part de cet angle à tous les 1/60 ième de seconde.  

```python
    ball.angle += 200 * dt
```
    
Comme la palette précédemment, la balle a une vitesse mais cette fois la vitesse est sous forme horizontale et verticales contrairement à la palette qui avant seulement une vitesse verticale.

Donc le déplacement de la balle se calcule par sa position précédente + la vitesse * par le temps.

**nouvelle position = position actuelle + vitesse * temps**

On calcule le déplacement de la balle en horizontal selon la vitesse de la balle

Celle-ci est la position de la balle en horizontale recalculé avec le nouveau cycle

```python
    ball.x += state.bvx * dt
```
Celle-ci est la position de la balle en verticale recalculé avec le nouveau cycle

```python
    ball.y += state.bvy * dt
```

### Balle trop haute
Si la balle atteint le haut du jeu, on doit la retourner dans le jeu en calculant l'angle de retour, on ramène la balle à la position maximum du jeu (pas à l'extérieur) on inverse sa velocité de la balle pour qu'elle descende **ball.top >= app.height**


```python
    if ball.top >= app.height:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.top - app.height
        state.bvy = -state.bvy
```

Cette instruction permet de jouer un son.
```python
    pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
```

### Balle trop basse
Si la balle est plus basse que le jeu **ball.bottom <= 0**
Si la balle a atteint le bas du jeu, on la retourne comme ci-haut en inversant la vitesse verticale de la balle **bvy** pour qu'elle remonte

```python
    if ball.bottom <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.bottom
        state.bvy = -state.bvy
```

### Balle trop à droite

Si la balle dépasse à droite, on ramène la composant horizontal de la balle (x) dans le jeu, ensuite il faut changer la vélocité de X pour la retourner dans l'autre sens car la balle va suivre une direction inverse (elle rebondit)    

On va inverser la vélocité horizontale (x) car la balle passe de vers la droite à vers la gauche.  Pour la vélocité verticale, on garde la même direction.  Donc la vitesse choisie (192 valeur arbitraire choisie initialement au début du jeu) conserve le même signe en verticale et change de signe en horizontal. 

```python
    if ball.right >= app.width:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.right - app.width
        
        state.bvx = -192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
```

Enfin, il faut aussi donner un point à l'autre joueur, celui de gauche, si la balle se rend à droite complètement, **state.sl** et changer le texte du label **scorel**

```python
        state.sl += 1
        scorel.text = str(state.sl)
```

### Balle trop à gauche

Pour la gauche, c'est pratiquement le même code, si la balle dépasse à droite, on ramène le composant horizontal de la balle (x) dans le jeu. On inverse le signe en horizontale de la vélocité de la balle (bvx) et on garde le même signe en vertical de la vélocité de la balle (bvy), là-aussi le pointage augmente, mais cette fois-ci, pour l'autre joueur.

```python   
    if ball.left <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.left
        
        state.bvx = 192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sr += 1
        scorer.text = str(state.sr)
```   
 
### Balle proche de la palette droite 

Si la balle est à la même hauteur que la palette **ball.top >= padr.bottom and padr.top >= ball.bottom**
On vérifie si la distance entre la balle et la palette est sous 10 pixels **0 < ball.right - padr.left < 10**

Ici on va regarder si la palette est proche de la balle du côté droit du jeu.   Si c'est le cas, on repositionne la balle pour la coller sur la pallette, on inverse la vitesse horizontale de la balle **bvx** et on augmente la vitesse de la balle de 50% de la vitesse de la palette. Ceci est pour simuler qu'un effet de vitesse augmente la vitesse de la balle, dans la réalité, c'est plutôt une vitesse de rotation de la balle qui augmenterait et on aurait un effet de courbe comme lorsque l'on joue au squash et au tennis. 

```python   
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x -= ball.right - padr.left
            state.bvx = -state.bvx
            state.bvy += state.vyr / 2
```   
             

### Balle proche de la palette gauche 

Maintenant on s'occupe du côté gauche du jeu et on fait sensiblement le même traitement pour la balle.  

```python  
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x += ball.left - padl.right
            state.bvx = -state.bvx
            state.bvy += state.vyl / 2
```   
 
### Repositionnement de la vitesse de la balle en horizontal

Maintenant on recalcule la vitesse totale en horizontal et vertical, le même calcul qu'on avant fait au début de la fonction.        
            
 ```python   
    bs1 = state.bvx ** 2 + state.bvy ** 2
 ```
Ici on ajuste la vitesse de la balle horizontale si elle est trop basse par rapport au début de la fonction

Si le nouveau calcul de vitesse est plus petit de 90% de la vitesse avant le réajustement de la vitesse dans cette fonction
On recalcule la vitesse en horizontal sans tenir compte de la coposante de vitesse verticale **state.bvy*
En gros on ramène la vitesse horizontale **bvx** à son origine avant l'exécution de la fonction 

```python   
    if bs1 < 0.9 * bs0:
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)
```


Enfin, on s'assure que la balle reste dans le cadre du jeu

```python   
    ball.wrap_position(app.width, app.height)
```



In [ ]:
@app.run_me_every(1/60)
def update_ball(ct, dt):
    
    bs0 = state.bvx ** 2 + state.bvy ** 2    # premier calcul de la vitesse horizontale et verticale cumulée
    
    ball.angle += 200 * dt                   # changement de l'angle de la balle (carrée) en proportion de 1/60 seconde 
    
    ball.x += state.bvx * dt                 # déplacement horizontal de la balle sur 1/60 seconde
    ball.y += state.bvy * dt                 # déplacement vertical de la balle sur 1/60 seconde   
    
    if ball.top >= app.height:                                              # vérification si la balle est trop haute 
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note  
        ball.y -= ball.top - app.height                                     # on reposition la balle dans la limite du jeu
        state.bvy = -state.bvy                                              # on inverse la vitesse pour faire un changement de direction en vertical seulement
        
    if ball.bottom <= 0:                                                    # vérification si la balle est trop basse
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note     
        ball.y -= ball.bottom                                               # on ramette la balle dans les limites du jeu
        state.bvy = -state.bvy                                              # on inverse la vitesse pour faire un changement de direction en vertical seulement    
        
    if ball.right >= app.width:                                             # vérification si la balle est trop à droite
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note
        ball.x -= ball.right - app.width                                    # on repositionne la balle dans le cadre du jeu
        
        state.bvx = -192                                                    # on change la direction horizontale de la vitesse de la balle et on la met à 192
        state.bvy = 192 * np.sign(state.bvy)                                # on assigne la vitesse de la balle à 192 sans changer la direction en verticale
        bs0 = 0                                                             # on met la vitesse initiale cumulée a 0 lorsque la limite du jeu est atteinte
        
        state.sl += 1                                                       # on donne un point au joueur de gauche car la balle a atteint la limite droite
        scorel.text = str(state.sl)                                         # on affiche le score sur le plateau de jeu du joueur de gauche    
        
    if ball.left <= 0:                                                  
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)             # On vérifie si la balle est trop à gauche
        ball.x -= ball.left                                                         # on reposition la balle dans la limite du jeu
        
        state.bvx = 192                                                             # on applique la vitesse horizontale de la balle à la valeur par défaut en positif pour que la direction soit vers la droite. 
        state.bvy = 192 * np.sign(state.bvy)                                        # on applique la vitesse verticale de la balle à la valeur par défaut sans changer le signe de la direction verticale
        bs0 = 0                                                                     # on met la vitesse initiale cumulée a 0 lorsque la limite du jeu est atteinte
        
        state.sr += 1                                                               # on donne un point au joueur de droite car la balle a toucher a la limite gauche
        scorer.text = str(state.sr)
        
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom:       # on vérifie si la balle est dans les limite verticale de la palette de droite
        if 0 < ball.right - padr.left < 10:                                         # on vérifie si la balle est 10 pixels ou moins près de la palette de droite     
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)         # on joue un son
            ball.x -= ball.right - padr.left                                        # on position la balle sur la palette de droite
            state.bvx = -state.bvx                                                  # on inverse la vitesse horizontale de la balle pour faire un changement de direction
            state.bvy += state.vyr / 2                                              # on ajuste la vitesse verticale de la balle en lui ajoutant 50% de la vitesse de la palette droite
            
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom:       # on vérifie si la balle est dans les limite verticale de la palette de gauche
        if 0 < padl.right - ball.left < 10:                                         # on vérifie si la balle est 10 pixels ou moins près de la palette de gauche
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)         # on joue un son    
            ball.x += ball.left - padl.right                                        # on position la balle sur la palette de gauche
            state.bvx = -state.bvx                                                  # on inverse la vitesse horizontale de la balle pour faire un changement de direction
            state.bvy += state.vyl / 2                                              # on ajuste la vitesse verticale de la balle en lui ajoutant 50% de la vitesse de la palette droite
            
    bs1 = state.bvx ** 2 + state.bvy ** 2                                           # deuxième calcul de la vitesse horizontale et verticale cumulée
    
    if bs1 < 0.9 * bs0:                                                             # Si le deuxième calcul est plus petit que 90% de la vitesse cumulée, on recalcul la vitesse de la balle en horizontale pour remettre la vitesse horizontale initiale
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)              

    ball.wrap_position(app.width, app.height)                                       # on s'assure de ne pas dessiner la balle hors des limites du jeu    

# Etape 3: Jeu complet avec déplacement des palettes et la balle

In [ ]:
import logging
import sys
import os

import numpy as np


sys.path.insert(0, os.path.abspath('./..'))

import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

app = App()#log_level=logging.INFO)

background = '#3e32a2'
foreground = '#7c71da'

a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255


pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()

ball = Sprite(a0, y=app.height/2, x=app.width/2)

padl = Sprite(a1, y=app.height/2, x=48)
padr = Sprite(a1, y=app.height/2, x=app.width-48)

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

scorel = Label(
    '0', font_size=42, color=foreground, 
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground, 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)


@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()


state = State(
    
    sl = 0,
    sr = 0,
    
    bvx = 192,
    bvy = 192,
    
    vyl = 250,
    pyl = app.height/2,

    vyr = 250,
    pyr = app.height/2,

    left = False,
    right = False,

    key_a = False,
    key_d = False,
)    


@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:
        
        if key == keys.LEFT:
            state.left = True

        if key == keys.RIGHT:
            state.right = True

        if key == keys.A:
            state.key_a = True

        if key == keys.D:
            state.key_d = True

    if action == keys.ACTION_RELEASE:

    
        if key == keys.LEFT:
            state.left = False

        if key == keys.RIGHT:
            state.right = False

        if key == keys.A:
            state.key_a = False

        if key == keys.D:
            state.key_d = False


@app.run_me_every(1/60)
def update_pads(ct, dt):
    moveL = 0                                                 # Initialisation du deplacement de la palette gauche
    moveR = 0                                                 # Initialisation du deplacement de la palette droite   
    if state.right:
        moveR = 1                                             # Déplacement positif de la palette droite (vers vers le haut)  
        
    if state.left:
        moveR = -1                                            # Déplacement negatif de la palette droite (vers le bas)  

    if state.key_a:
        moveL = 1                                             # Déplacement positif de la palette gauche (vers le haut)  

    if state.key_d:
        moveL = -1                                            # Déplacement negatif de la palette droite (vers le bas)  

    if moveL != 0:
        padl.y += moveL * state.vyl * dt                      # calcul de la position de la palette gauche
    if moveR != 0:
        padr.y += moveR * state.vyr * dt                      # calcul de la position de la palette droite
          
    
    padr.clip_position(app.width, app.height)                 # Conserver l'image de la palette droite dans le cadre du jeu
    padl.clip_position(app.width, app.height)                 # Conserver l'image de la palette gauche dans le cadre du jeu



@app.run_me_every(1/60)
def update_ball(ct, dt):
    
    bs0 = state.bvx ** 2 + state.bvy ** 2
    
    ball.angle += 200 * dt
    
    ball.x += state.bvx * dt
    ball.y += state.bvy * dt
    
    if ball.top >= app.height:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.top - app.height
        state.bvy = -state.bvy
        
    if ball.bottom <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.bottom
        state.bvy = -state.bvy
        
    if ball.right >= app.width:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.right - app.width
        
        state.bvx = -192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sl += 1
        scorel.text = str(state.sl)
        
     # + ' ' + state.pyr + ' ' + padl.y    
    if ball.left <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.left
        
        state.bvx = 192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sr += 1
        scorer.text = str(state.sr)
        
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x -= ball.right - padr.left
            state.bvx = -state.bvx
            state.bvy += state.vyr / 2
            
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x += ball.left - padl.right
            state.bvx = -state.bvx
            state.bvy += state.vyl / 2
            
    bs1 = state.bvx ** 2 + state.bvy ** 2
    
    if bs1 < 0.9 * bs0:
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)

    ball.wrap_position(app.width, app.height)


app.run()